In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("../src")

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from data_processing.geometry_processor import GeometryProcessor
from data_processing.band_dto import BandDTO
from data_processing.feature_service import FeatureService
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans, kmeans_plusplus

In [ ]:
gp = GeometryProcessor()
band_data  = gp.flatten_and_filter_monthly_data()

In [ ]:
fs = FeatureService(band_data)
feature_df = fs.calculate_features_for_monthly_data()

# feature_df.to_csv("features.csv")

In [ ]:
scaler = MinMaxScaler()
feature_norm = scaler.fit_transform(feature_df)

pca = PCA(n_components=3)
X_transformed = pca.fit_transform(feature_norm)

In [ ]:
plot_data = X_transformed[::50].T
kde = gaussian_kde(plot_data)
density = kde(plot_data)

fig = go.Figure(data=[go.Scatter3d(
    x=X_transformed[::50, 0],
    y=X_transformed[::50, 1],
    z=X_transformed[::50, 2],
    mode='markers',
    marker=dict(
        size=4,
        color=density,
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title='Density')
    )
)])

fig.update_layout(
    title='3D Density Scatter Plot',
    scene=dict(
        xaxis_title='PCA-Component 1',
        yaxis_title='PCA-Component 2',
        zaxis_title='PCA-Component 3'
    ),
    margin=dict(l=0, r=0, b=20, t=40),
    width=800,
    height=600
)

fig.show()

In [ ]:
pca.components_

In [ ]:
sse = []

for i in range(1, 10):
    centroids, _ = kmeans_plusplus(feature_norm, n_clusters=i)
    kmeans = KMeans(n_clusters=i, random_state=10, init=centroids)

    labels = kmeans.fit_predict(feature_norm)
    
    sse.append(kmeans.inertia_)

In [ ]:
fig, ax = plt.subplots()

ax.plot(range(1, 10), sse)

In [ ]:
n_clusters = 4
centroids, _ = kmeans_plusplus(feature_norm, n_clusters=n_clusters, random_state=20)
kmeans = KMeans(n_clusters=n_clusters, random_state=20, init=centroids)

labels = kmeans.fit_predict(feature_norm)

In [ ]:
labels_2d = gp.reconstruct_2d(labels)
gp.export_reconstruction_as_geotiff(labels, "labels.tif")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(labels_2d, cmap='viridis')
ax.set_title('Cluster Labels')

cbar = fig.colorbar(im, ax=ax, shrink=0.5)
cbar.set_label("Cluster ID")

plt.tight_layout()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=X_transformed[::50, 0],
    y=X_transformed[::50, 1],
    z=X_transformed[::50, 2],
    mode='markers',
    marker=dict(
        size=4,
        color=labels[::50],
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title='Cluster Label')
    )
)])

fig.update_layout(
    title='3D Cluster Scatter Plot',
    scene=dict(
        xaxis_title='PCA-Component 1',
        yaxis_title='PCA-Component 2',
        zaxis_title='PCA-Component 3'
    ),
    margin=dict(l=0, r=0, b=20, t=40),
    width=800,
    height=600
)

fig.show()

In [ ]:
result_df = feature_df.copy()
result_df["cluster"] = labels

In [ ]:
result_df.columns

In [ ]:
y_cols = [
    "mean",
    "mean2",
    "difference_in_mean_between_intervals",
    "std",
    "spatial_std_difference",
]

titles = {
    "mean": "Mean NDVI (Last 12 Months)",
    "mean2": "Mean NDWI (Last 12 Months)",
    "difference_in_mean_between_intervals": "NDWI Change: Recent 12 Months vs First 12 Months",
    "std": "Temporal Variability of SAVI",
    "spatial_std_difference": "Change in Local NDVI Spatial Variability (5×5 Window)"
}


fig, axes = plt.subplots(1, len(y_cols), figsize=(20, 4), sharex=True)

for ax, col in zip(axes, y_cols):
    sns.boxplot(data=result_df, x="cluster", y=col, ax=ax)
    ax.set_title(titles[col])

plt.tight_layout()
plt.show()


![Result](../DATA/images/waldmonitor.png)  
Source: https://map3d.remote-sensing-solutions.de/waldmonitor-deutschland/# (16.02.2026)

![deadtrees](../DATA/images/deadtrees.png)  
Source: https://deadtrees.earth/deadtrees (16.02.2026)